In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import os

In [6]:
# This shows how to save/restore your model (trained variables).
# To see how it works, please stop this program during training and resart.
# This network is the same as 3_net.py

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden): # this network is the same as the previous one except with an extra hidden layer + dropout
    X = tf.nn.dropout(X, p_keep_input)
    h = tf.nn.relu(tf.matmul(X, w_h))

    h = tf.nn.dropout(h, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h, w_h2))

    h2 = tf.nn.dropout(h2, p_keep_hidden)

    return tf.matmul(h2, w_o)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
X = tf.placeholder("float", [None, 784])
Y = tf.placeholder("float", [None, 10])

w_h = init_weights([784, 625])
w_h2 = init_weights([625, 625])
w_o = init_weights([625, 10])

p_keep_input = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

ckpt_dir = "./ckpt_dir"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

global_step = tf.Variable(0, name='global_step', trainable=False)

# Call this after declaring all tf.Variables.
saver = tf.train.Saver()

# This variable won't be stored, since it is declared after tf.train.Saver()
non_storable_variable = tf.Variable(777)

In [8]:
# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.initialize_all_variables().run()

    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print(ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path) # restore all variables

    start = global_step.eval() # get last global_step
    print("Start from:", start)

    for i in range(start, 100):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_input: 0.8, p_keep_hidden: 0.5})

        global_step.assign(i).eval() # set and update(eval) global_step with index, i
        saver.save(sess, ckpt_dir + "/model.ckpt", global_step=global_step)
        print(i, np.mean(np.argmax(teY, axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX, Y: teY,
                                                         p_keep_input: 1.0,
                                                         p_keep_hidden: 1.0})))

Start from: 0
0 0.9269
1 0.9611
2 0.9662
3 0.9738
4 0.9742
5 0.9765
6 0.9762
7 0.9785
8 0.9797
9 0.9803
10 0.9792
11 0.9806
12 0.9807
13 0.9814
14 0.9823
15 0.9818
16 0.982
17 0.9836
18 0.9798
19 0.9817
20 0.983
21 0.9823
22 0.9835
23 0.9817
24 0.9828
25 0.9823
26 0.9838
27 0.9832
28 0.9832
29 0.9832
30 0.9835
31 0.9835
32 0.9812
33 0.9841
34 0.9839
35 0.9836
36 0.9829
37 0.9834
38 0.9844
39 0.983
40 0.9849
41 0.9841
42 0.9848
43 0.9836
44 0.9848
45 0.9846
46 0.9839
47 0.986
48 0.9848
49 0.9844
50 0.9839
51 0.9847
52 0.9841
53 0.984
54 0.9847
55 0.9838
56 0.9845
57 0.9854
58 0.9842
59 0.9843
60 0.9834
61 0.9853
62 0.9843
63 0.9852
64 0.9844
65 0.9843
66 0.9846
67 0.9847
68 0.9834
69 0.985
70 0.9846
71 0.9843
72 0.9843
73 0.9854
74 0.9849
75 0.985
76 0.9844
77 0.9858
78 0.9854
79 0.9846
80 0.9847
81 0.9849
82 0.9848
83 0.9844
84 0.9856
85 0.9852
86 0.985
87 0.9845
88 0.9861
89 0.9847
90 0.9843
91 0.9859
92 0.9846
93 0.9845
94 0.9852
95 0.9851
96 0.9845
97 0.9845
98 0.9856
99 0.9855
